<a href="https://colab.research.google.com/github/akashban812/DS-practice/blob/main/Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import os as os
os.chdir('/content/drive/MyDrive/DS notebooks')
os.getcwd()
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [25]:
#@title Default title text
##Importing the data
##Unicode escape in activated to encounter any error arising due to any invalid characters

## Using the 'Online Retail' dataset from Kaggle


rtl_raw_data=pd.read_csv('OnlineRetail.csv',encoding='unicode_escape')



In [26]:
###checking distribution by country
country_cust=rtl_raw_data[['Country','CustomerID']].drop_duplicates()
country_cust_grp= country_cust.groupby(['Country'])['CustomerID'].nunique().reset_index().sort_values(by=['CustomerID'],ascending=[False])
country_cust_grp

##In the next step we'll use only the records for UK as it has the max customers
rtl_raw_UK=rtl_raw_data[rtl_raw_data['Country']=='United Kingdom'].reset_index(drop=True)

In [27]:

##Checking for missng values
rtl_raw_UK.isnull().sum(axis=0)



InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [28]:
##removing missing values for the customerid column
rtl_raw_UK_2=rtl_raw_UK[pd.notnull(rtl_raw_UK['CustomerID'])]

##Validating if there are any negative vales in the Quantity column
rtl_raw_UK_2.Quantity.min()


-80995

In [29]:
##Validating if there are any negative vales in the UnitPrice column
rtl_raw_UK_2.UnitPrice.min()

0.0

In [30]:
##Filtering out negative values from Quantity column
rtl_raw_UK_3=rtl_raw_UK_2[rtl_raw_UK_2['Quantity']>0]

##Converting the date filed to datetime
rtl_raw_UK_3['InvoiceDate']=pd.to_datetime(rtl_raw_UK_3['InvoiceDate'])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [31]:
###Calculation of total amt
rtl_raw_UK_3['total_spend']=rtl_raw_UK_3['Quantity']*rtl_raw_UK_3['UnitPrice']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### From this point onwards, we demonstrate RFM modeling using RFM scores


In [36]:
import datetime as dt
### Setting latest date as per dataset
latest_dt=dt.datetime(2011,12,10)

###Creating RFM scores for each customer

rfm_scores= rtl_raw_UK_3.groupby(['CustomerID']).agg({'InvoiceDate': lambda x: (latest_dt - x.max()).days,\
                                                      'InvoiceNo': lambda x: len(x), 'total_spend': lambda x: x.sum()}).reset_index()

rfm_scores.rename(columns={'InvoiceDate':'recency','InvoiceNo':'frequency','total_spend':'monetary'},inplace=True)
rfm_scores

,CustomerID,recency,frequency,monetary
0,12346.0,325,1,77183.60
1,12747.0,22,103,4196.01
2,12748.0,4,4596,33719.73
3,12749.0,22,199,4090.88
4,12820.0,44,59,942.34
...,...,...,...,...
3916,18280.0,159,10,180.60
3917,18281.0,3,7,80.82
3918,18282.0,215,12,178.05
3919,18283.0,9,756,2094.88


In [38]:
###Calculating RFM groups

labels_rec = [1,2,3,4]
labels_fm = [4,3,2,1]

rfm_scores['rec_score']=pd.qcut(rfm_scores['recency'],q=[0,0.25,0.5,0.75,1],labels=labels_rec)
rfm_scores['freq_score']=pd.qcut(rfm_scores['frequency'],q=[0,0.25,0.5,0.75,1],labels=labels_fm)
rfm_scores['mon_score']=pd.qcut(rfm_scores['monetary'],q=[0,0.25,0.5,0.75,1],labels=labels_fm)


rfm_scores['rfm_groups']= rfm_scores['rec_score'].astype(str) + rfm_scores['freq_score'].astype(str) + rfm_scores['mon_score'].astype(str)
rfm_scores['rfm_score']= rfm_scores[['rec_score','freq_score','mon_score']].sum(axis=1)




In [39]:
loyalty_level=['Platinum','Gold','Silver','Bronze']

rfm_scores['loyalty']=pd.qcut(rfm_scores['rfm_score'],q=[0,0.25,0.5,0.75,1],labels=loyalty_level)

rfm_scores

### Concludes the RFM scores methodology

,CustomerID,recency,frequency,monetary,rec_score,freq_score,mon_score,rfm_groups,rfm_score,loyalty
0,12346.0,325,1,77183.60,4,4,1,441,9,Silver
1,12747.0,22,103,4196.01,2,1,1,211,4,Platinum
2,12748.0,4,4596,33719.73,1,1,1,111,3,Platinum
3,12749.0,22,199,4090.88,2,1,1,211,4,Platinum
4,12820.0,44,59,942.34,2,2,2,222,6,Gold
...,...,...,...,...,...,...,...,...,...,...
3916,18280.0,159,10,180.60,3,4,4,344,11,Bronze
3917,18281.0,3,7,80.82,1,4,4,144,9,Silver
3918,18282.0,215,12,178.05,4,4,4,444,12,Bronze
3919,18283.0,9,756,2094.88,1,1,1,111,3,Platinum


In [ ]:
### This portion will cover the K-Means clustering
